In [74]:
# Code based on example from: https://realpython.com/async-io-python/

import asyncio
import logging
import re
import sys
from typing import IO
import urllib.error
import urllib.parse

import aiofiles
import aiohttp
from aiohttp import ClientSession

import pandas as pd
from tqdm.notebook import tqdm

# from throttler import Throttler

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(name)s: %(message)s",
    level=logging.DEBUG,
    datefmt="%H:%M:%S",
    stream=sys.stderr,
)
logger = logging.getLogger("areq")
logging.getLogger("chardet.charsetprober").disabled = True

ADMIN_RE = re.compile(r'<adminEmail>([^<]+)<')
REPO_RE = re.compile(r'<repositoryName>([^<]+)<')
TIME_RE = re.compile(r'<earliestDatestamp>([^<]+)<')

async def fetch_xml(url: str, session: ClientSession, **kwargs) -> str:
    """GET request wrapper to fetch page HTML.

    kwargs are passed to `session.request()`.
    """    
    resp = await session.request(method="GET", url=url, **kwargs)
    resp.raise_for_status()
#     logger.info("Got response [%s] for URL: %s", resp.status, url)
    xml = await resp.text()
    return xml

async def process_url(url: str, session: ClientSession, **kwargs) -> set:
    """Find HREFs in the HTML of `url`."""
#     found = set()
    try:
        xml = await fetch_xml(url=url, session=session, **kwargs)
    except (
        aiohttp.ClientError,
        aiohttp.http_exceptions.HttpProcessingError,
    ) as e:
        logger.error(
            "aiohttp exception for %s [%s]: %s",
            url,
            getattr(e, "status", None),
            getattr(e, "message", None),
        )
        return False
    except Exception as e:
        logger.exception(
            "Non-aiohttp exception occured:  %s", getattr(e, "__dict__", {})
        )
        return False
    else:
        email = repo = time = 'notFound'
        try: 
            email = ADMIN_RE.search(xml).group(1)
        except:
            pass
        try: 
            repo = REPO_RE.search(xml).group(1)
        except:
            pass
        try: 
            time = TIME_RE.search(xml).group(1)
        except:
            pass        
#         logger.info("Found %d links for %s", len(found), url)
        return '"' + '","'.join([email,repo,time]) + '"'

async def write_one(file: IO, url: str, **kwargs) -> None:
    """Write the found HREFs from `url` to `file`."""
    res = await process_url(url=url, **kwargs)
    if not res:
        return None
    async with aiofiles.open(file, "a") as f:
        await f.write(f"{url},{res}\n")
        logger.info("Wrote results for source URL: %s", url)

# async def bulk_crawl_and_write(file: IO, urls: set, **kwargs) -> None:
#     """Crawl & write concurrently to `file` for multiple `urls`."""
#     async with ClientSession() as session:
#         tasks = []
#         for url in urls:
#             tasks.append(
#                 write_one(file=file, url=url, session=session, **kwargs)
#             )
#         await asyncio.gather(*tasks)

async def bulk_crawl_and_write(file: IO, urls: set, **kwargs) -> None:
        connector = aiohttp.TCPConnector(limit=100)
        timeout = aiohttp.ClientTimeout(total=None)
        async with aiohttp.ClientSession(
            connector=connector, timeout=timeout
        ) as session:
            # Create tasks with DOI
            tasks = []
#             for doi in dois:
#                 t = asyncio.ensure_future(self.get_pmid(session, doi, writer))
#                 tasks.append(t)
            for url in urls:
                tasks.append(
                    write_one(file=file, url=url, session=session, **kwargs)
                )                

            # Await tasks and print progress for responses
            for t in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
                await t
        
import pathlib
import sys


# urls = ['http://publikationen.soziologie.de'] # 'http://homologa.publicacoes.sei.ba.gov.br/index.php/index/oai', 'https://revistas.sopcom.pt/index.php/index/oai', 'http://abdiputra.nusaputra.ac.id/index.php/index/oai', 'http://adrjournalshouse.com/index.php/index/oai', 'https://papiro.unizar.es/ojs/index.php/index/oai', 'https://journals.tdl.org/pal/index.php/index/oai', 'http://revistawiphala.org/index.php/index/oai', 'https://sttexcelsius.ac.id/e-journal/index.php/index/oai', 'https://e-journal.fh.unmul.ac.id/index.php/index/oai', 'https://www.scpe.org/index.php/index/oai']
# urls = ['http://publikationen.soziologie.de/index.php/index/oai', 'http://publikationen.soziologie.de/index/oai', 'http://publikationen.soziologie.de/index.php?page=oai', 'http://publikationen.soziologie.de?page=oai']

outpath = "data/foundoai.txt"
with open(outpath, "w") as outfile:
    outfile.write("oai_url,contact,name,time\n")



In [27]:
def domain_filter(url):
    ''' Don't consider domains that are IP addresses or are 'localhost'
        Consider also filtering things with 'demo' or 'test'
    '''
    
    domain = urllib.parse.urlparse(url).netloc
    if 'localhost' in domain: 
        return False
    if domain.replace('.', '').replace(':', '').isnumeric() == True:
        return False
    return True

ojs_in_logs = pd.read_csv('data/2020/ojs_in_logs.csv')
ojs_in_logs = ojs_in_logs[ojs_in_logs.base_url.map(domain_filter)]    

In [6]:
url = url.strip("/")
verb_param = {'verb': 'Identify'}
urls_to_try = []  # all the combinations of possible OAI URLs, given the base url
urls_to_try.append(url + "/index.php/index/oai")
urls_to_try.append(url + "/index/oai")
urls_to_try.append(url + "/index.php?page=oai")
urls_to_try.append(url + "?page=oai")


'http://portaldoenvelhecimento.org.br/revista-nova'

### re-running the same things with the various OAI urls to make sure we get thm all (excluding those for which we alrady succeeded)

In [ ]:
urls = ojs_in_logs.base_url.map(lambda x: x + "/index.php/index/oai").to_list()
await bulk_crawl_and_write(file=outpath, urls=urls, params={'verb': 'Identify'})

In [ ]:
urls = set(ojs_in_logs.base_url).difference(df.source_url.map(lambda u: u[:-len('/index.php/index/oai')]).to_list())
urls = [u + "/index/oai" for u in urls]
await bulk_crawl_and_write(file=outpath, urls=urls, params={'verb': 'Identify'})

In [ ]:
urls = set(ojs_in_logs.base_url).difference(df.source_url.map(lambda u: u.replace('/index.php/index/oai', '').replace('/index/oai', '')).to_list())
urls = [u + "/index.php?page=oai" for u in urls]
await bulk_crawl_and_write(file=outpath, urls=urls, params={'verb': 'Identify'})

In [58]:
df = pd.read_csv(outpath)
del df['found_oai']

def remove_oai(url):
    return url.replace('/index.php/index/oai', '').replace('/index/oai', '').replace('/index.php?page=oai', '')

df.loc[:,'base_url'] = df.source_url.map(remove_oai)
df.columns = ['oai_url', 'base_url']
df.to_csv(outpath, index=False)

In [119]:
ojs_in_tracker = pd.read_csv('data/2020/ids_in_tracker.csv')
ojs_in_tracker.set_index('oai_url', inplace=True)
ojs_in_tracker.columns = ['date_in_beacon', 'beacon_id']
ojs_in_tracker.head()

,date_in_beacon,beacon_id
oai_url,,
:///bjpa.bpatc.org.bd/oai,2020-07-19 06:20:06,5da7eff30aedd
:///https//jurnal.polgan.ac.id/oai,2020-05-25 10:18:19,58ebd67238c9d
:///num-meth.rcc.msu.ru/oai,2020-07-26 23:15:01,5d668afcb48f4
http:///revistas.uazuay.edu.ec/index/oai,2020-07-26 08:12:33,5a1cac081d4e1
http://0nlineastrology.com/index.php/index/oai,2020-07-18 00:41:21,5f12a61aafaca


In [59]:
ojs_in_db = pd.read_csv('data/2020/ojs_in_db.csv')
ojs_in_db.loc[:,'oai_url'] = ojs_in_db.oai_url.map(lambda x: x.replace("verb=identify", "").strip('&?'))

ojs_in_logs = pd.read_csv(outpath)

In [75]:
oai_urls = list(set(ojs_in_db.oai_url).union(ojs_in_tracker.oai_url).union(ojs_in_logs.oai_url))
await bulk_crawl_and_write(file=outpath, urls=oai_urls, params={'verb': 'Identify'})

18:12:05 ERROR:asyncio: Task was destroyed but it is pending!
task: <Task pending name='Task-21478' coro=<write_one() done, defined at <ipython-input-2-c92f0d27ce95>:77> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x11de15310>()]> cb=[as_completed.<locals>._on_completion() at /usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/tasks.py:595]>


18:12:06 ERROR:areq: aiohttp exception for :///https//jurnal.polgan.ac.id/oai [None]: None
18:12:09 ERROR:areq: aiohttp exception for http:///revistas.uazuay.edu.ec/index/oai [None]: None
18:12:09 ERROR:areq: aiohttp exception for :///num-meth.rcc.msu.ru/oai [None]: None
18:12:12 ERROR:areq: aiohttp exception for :///bjpa.bpatc.org.bd/oai [None]: None
18:12:14 ERROR:areq: aiohttp exception for http://www.zwgm.org/index.php/index/oai [None]: None
18:12:14 ERROR:areq: aiohttp exception for https://svkooaptest01.gesis.intra/mdatest/index.php/index/oai [None]: None
18:12:14 ERROR:areq: Non-aiohttp exception occured:  {}
Traceback (most recent call last):
  File "<ipython-input-74-8e158baf2102>", line 48, in process_url
    xml = await fetch_xml(url=url, session=session, **kwargs)
  File "<ipython-input-74-8e158baf2102>", line 41, in fetch_xml
    xml = await resp.text()
  File "/usr/local/lib/python3.8/site-packages/aiohttp/client_reqrep.py", line 1014, in text
    return self._body.decode

18:12:35 INFO:areq: Wrote results for source URL: http://nazharat.fah.uinjambi.ac.id/index.php/index/oai
18:12:35 INFO:areq: Wrote results for source URL: https://njas.fi/index/oai
18:12:36 INFO:areq: Wrote results for source URL: http://site.veracruz.edu.br/instituto/revistaveras/index.php/index/oai
18:12:36 INFO:areq: Wrote results for source URL: http://ijasm.altervista.org/index.php/index/oai
18:12:36 INFO:areq: Wrote results for source URL: http://qu.edu.iq/journalsc/index.php/index/oai
18:12:36 INFO:areq: Wrote results for source URL: https://jurnalkesehatan.unisla.ac.id/index.php/index/oai
18:12:36 ERROR:areq: aiohttp exception for http://www.cbmh.ca/index.php/index/oai [404]: Not Found
18:12:36 ERROR:areq: aiohttp exception for http://www.jurnal.poltekkes-soepraoen.ac.id/index.php/index/oai [None]: None
18:12:36 INFO:areq: Wrote results for source URL: https://ejournal.ijshs.org/index.php/index/oai
18:12:36 INFO:areq: Wrote results for source URL: http://davarlogos.uap.edu.ar/i

18:12:47 ERROR:areq: aiohttp exception for http://www.aibr.org/oj/index.php/index/oai [404]: Not Found
18:12:47 ERROR:areq: aiohttp exception for http://rdl.org.br/seer/index.php/index/oai [None]: None
18:12:47 ERROR:areq: aiohttp exception for http://fabemarau.edu.br/seer2/index.php/index/oai [404]: Not Found
18:12:47 ERROR:areq: aiohttp exception for http://surversion.celarg.gob.ve/index.php/index/oai [None]: None
18:12:48 ERROR:areq: aiohttp exception for http://adlibilimler.xyz/index.php/index/oai [None]: None
18:12:48 INFO:areq: Wrote results for source URL: http://www.catalysis-kalvis.ru/index.php/index/oai
18:12:48 INFO:areq: Wrote results for source URL: http://ojs.uma.ac.id/index.php/index/oai
18:12:48 INFO:areq: Wrote results for source URL: http://rnojournal.binghamton.edu/index.php/index/oai
18:12:48 INFO:areq: Wrote results for source URL: http://rjh.folium.ru/index.php/index/oai
18:12:48 INFO:areq: Wrote results for source URL: https://jeredajournal.com/index.php/index/oa

18:12:54 ERROR:areq: aiohttp exception for http://revistacis.techo.org/index.php/index/oai [None]: None
18:12:54 ERROR:areq: aiohttp exception for http://www.cra-pr.org.br/revista/index.php/index/oai [None]: None
18:12:54 INFO:areq: Wrote results for source URL: https://rejur.ufrrj.br/ojs/index.php/index/oai
18:12:54 INFO:areq: Wrote results for source URL: http://resocjalizacjapolska.pl/index.php/index/oai
18:12:54 INFO:areq: Wrote results for source URL: http://jkp.unram.ac.id/index.php/index/oai
18:12:54 INFO:areq: Wrote results for source URL: http://zelcart.com/index.php/index/oai
18:12:55 INFO:areq: Wrote results for source URL: http://www.pagepress.org/journals/index.php/index/oai
18:12:55 INFO:areq: Wrote results for source URL: http://www.vestnik.sci.pfu.edu.ru/index.php/index/oai
18:12:55 INFO:areq: Wrote results for source URL: http://ejournal.aibpm.org/index.php/index/oai
18:12:55 INFO:areq: Wrote results for source URL: http://eyad.mu.edu.tr/index.php/index/oai
18:12:55 IN

...

18:46:12 INFO:areq: Wrote results for source URL: http://spaj.ukm.my/apjmm/index.php/index/oai
18:46:12 INFO:areq: Wrote results for source URL: https://journals.iium.edu.my/ijes/index.php/index/oai
18:46:12 INFO:areq: Wrote results for source URL: https://journals.iium.edu.my/ijahs/index.php/index/oai
18:46:12 INFO:areq: Wrote results for source URL: http://www.unicap.br/ojs/////////////////index.php/index/oai
18:46:13 ERROR:areq: aiohttp exception for https://demos6.softaculous.com/Open_Journal_Systemsu46lvarkxq/index.php/index/oai [404]: Not Found
18:46:13 ERROR:areq: aiohttp exception for http://up.openjournaltheme.com/ecuatoriana/ojt_3124/index.php/index/oai [404]: Not Found
18:46:13 INFO:areq: Wrote results for source URL: https://journals.iium.edu.my/inst/index.php/index/oai
18:46:13 INFO:areq: Wrote results for source URL: https://journals.iium.edu.my/ktn/index.php/index/oai
18:46:13 INFO:areq: Wrote results for source URL: https://journals.iium.edu.my/iiibf-journal/index.php/i

18:46:18 ERROR:areq: aiohttp exception for http://journals.sfu.ca/paao/index.php/index/oai [500]: Internal Server Error
18:46:18 ERROR:areq: aiohttp exception for http://journals.sfu.ca/ccc/index.php/index/oai [404]: Not Found
18:46:18 ERROR:areq: aiohttp exception for http://journals.sfu.ca/ccsp/index.php/index/oai [404]: Not Found
18:46:18 INFO:areq: Wrote results for source URL: http://up.openjournaltheme.com/ojs3202/index.php/index/oai
18:46:18 INFO:areq: Wrote results for source URL: http://journals.sfu.ca/jd/index.php/index/oai
18:46:18 INFO:areq: Wrote results for source URL: http://journals.sfu.ca/iij/index.php/index/oai
18:46:18 INFO:areq: Wrote results for source URL: https://econtents.bc.unicamp.br/treina/index.php/index/oai
18:46:19 INFO:areq: Wrote results for source URL: http://journals.sfu.ca/flr/flr/flr/flr/index.php/index/oai
18:46:19 INFO:areq: Wrote results for source URL: http://journals.sfu.ca/rpfs/index.php/index/oai
18:46:19 ERROR:areq: aiohttp exception for http

18:46:27 INFO:areq: Wrote results for source URL: https://spektrum.stikosa-aws.ac.id/index.php/index/oai
18:46:28 ERROR:areq: aiohttp exception for http://mkweb.uni-pannon.hu/hjic/index.php/index/oai [None]: None
18:46:29 ERROR:areq: aiohttp exception for https://revues-travail.cirad.fr/index.php/index/oai [None]: None
18:46:29 INFO:areq: Wrote results for source URL: http://ojs.escire.net/ecn/index.php/index/oai
18:46:30 ERROR:areq: aiohttp exception for http://aula20.no-ip.org/ojs-2/index.php/index/oai [None]: None
18:46:30 INFO:areq: Wrote results for source URL: http://ojs.escire.net/ojs3/index.php/index/oai
18:46:31 ERROR:areq: aiohttp exception for http://journals.sfu.ca/ejl4d/index.php/index/oai [None]: None
18:46:31 INFO:areq: Wrote results for source URL: https://repa.jp/submit/rps/index.php/index/oai
18:46:31 ERROR:areq: aiohttp exception for http://inotera.poltas.ac.id/index.php/index/oai [None]: None
18:46:33 ERROR:areq: aiohttp exception for http://journal.aiou.edu.pk/pjeo

CancelledError: 

21:15:08 ERROR:areq: aiohttp exception for http://143.225.131.3/~ojs/index.php/index/oai [None]: None
21:15:08 ERROR:areq: aiohttp exception for http://astjournal.npru.ac.th/index.php/index/oai [None]: None
21:15:08 ERROR:areq: aiohttp exception for http://ejournal.stikespantirapih.ac.id/index.php/index/oai [None]: None
21:15:08 ERROR:areq: aiohttp exception for http://143.225.131.3/~rmojs/index.php/index/oai [None]: None
21:15:08 ERROR:areq: aiohttp exception for http://www.revistacoorte.com.br/index.php/index/oai [None]: None


In [107]:
df = pd.read_csv('data/foundoai.txt')
# df.to_csv('data/foundoai.txt', index=False)

In [130]:
# df = df[~((df.contact == 'notFound') & (df.name == 'notFound') & (df.time == 'notFound'))]

df.to_csv('data/foundoai.txt', index=False)

In [164]:
df = pd.read_csv('data/foundoai.txt')
df.set_index('oai_url', inplace=True)
df.fillna('', inplace=True)
df.loc[:,'makeshift_id'] = df[['contact', 'name', 'time']].apply(lambda row: ';'.join(row), axis=1)

In [165]:
df = df.join(ojs_in_tracker)

In [161]:
# df.groupby('makeshift_id').size().nlargest(40)

In [167]:
df.groupby('makeshift_id').beacon_id.idxmax()

ValueError: 